# TS2Vec sur ECG5000 — Evaluation
- Rechargement d'un run existant (EXP_ID choisi à la main)
- Extraction des représentations + logistic regression
- Affichage accuracies + quelques plots

In [ ]:
import os
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import models as m

from sktime.datasets import load_from_tsfile_to_dataframe
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Paths + run à recharger

In [ ]:
CURR_DIR = Path.cwd()
CURR_DIR_PARENT = list(CURR_DIR.parents)
SRC_DIR = CURR_DIR_PARENT[0]

DATA_DIR = SRC_DIR/ "data"
DATASET_DIR = DATA_DIR / "ECG5000"
train_path = DATASET_DIR / "ECG5000_TRAIN.ts"
test_path  = DATASET_DIR / "ECG5000_TEST.ts"

RUNS_DIR = CURR_DIR/"runs"

EXP_NAME = "ts2vec_ecg5000"

EXP_ID = "0"  

EXP_DIR = RUNS_DIR / EXP_NAME / EXP_ID
print("EXP_DIR:", EXP_DIR)

config_path = EXP_DIR / "config.json"
state_dict_path = EXP_DIR / "encoder.pt"

assert config_path.exists(), f"{config_path} not found"
assert state_dict_path.exists(), f"{state_dict_path} not found"

with open(config_path, "r") as f:
    config = json.load(f)

print("Config load:")
print(config)

## Data 

In [ ]:
X_train, y_train = load_from_tsfile_to_dataframe(str(train_path))
X_test,  y_test  = load_from_tsfile_to_dataframe(str(test_path))

print("X_train shape:", X_train.shape)
print("X_test shape :", X_test.shape)
print("Example series shape:", X_train.iloc[0, 0].shape)

def df_to_numpy(X):
    arr = np.stack([X.iloc[i, 0] for i in range(len(X))], axis=0)
    return arr[..., np.newaxis]  # (N, T, 1)

X_train_np = df_to_numpy(X_train)
X_test_np  = df_to_numpy(X_test)

X_train_torch = torch.from_numpy(X_train_np).float()
X_test_torch  = torch.from_numpy(X_test_np).float()
y_train_torch = torch.from_numpy(y_train.astype(np.int64))
y_test_torch  = torch.from_numpy(y_test.astype(np.int64))

train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset  = TensorDataset(X_test_torch,  y_test_torch)

train_eval_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_eval_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False)

## Modèle laoding

In [ ]:
device = "mps" if torch.mps.is_available() else "cpu"
print("Device:", device)

model = m.Encoder(
    in_channel=config["in_channel"],
    representation_dim=config["representation_dim"],
    num_blocks=config["num_blocks"],
    kernel_size=config["kernel_size"],
)
model.load_state_dict(torch.load(state_dict_path, map_location=device))
model.to(device)
model.eval()

total = sum(p.numel() for p in model.parameters())
print(f"Total params: {total/1e6:.2f} M")

In [ ]:
loss_path = EXP_DIR / "train_loss.npy"
if loss_path.exists():
    log_np = np.load(loss_path)
    plt.figure()
    plt.plot(log_np)
    plt.yscale("log")
    plt.title("Training loss (reloaded)")
    plt.xlabel("Iteration")
    plt.ylabel("HierLoss")
    plt.grid(True)
    plt.show()
else:
    print("No train_loss.npy found at", loss_path)

## Extraction des représentations + logistic regression

In [ ]:
@torch.no_grad()
def extract_representations(dataloader, model, device):
    zs = []
    ys = []
    for x, y in dataloader:
        # x: (B, T, 1) -> (B, 1, T)
        x = x.to(device).permute(0, 2, 1)

        r = model(x)  # (B, K, T)

        
        z = r.max(dim=-1).values # max pooling
      
        zs.append(z.cpu())
        ys.append(y.cpu())

    Z = torch.cat(zs, dim=0).numpy()   # (N, K)
    y = torch.cat(ys, dim=0).numpy()   # (N,)
    return Z, y

Z_train, y_train_np = extract_representations(train_eval_loader, model, device)
Z_test,  y_test_np  = extract_representations(test_eval_loader,  model, device)

print("Z_train / Z_test shapes:", Z_train.shape, Z_test.shape)

In [ ]:
clf = LogisticRegression(
    max_iter=1000,
    multi_class="auto"
)
clf.fit(Z_train, y_train_np)

y_pred_train = clf.predict(Z_train)
y_pred_test  = clf.predict(Z_test)

acc_train = accuracy_score(y_train_np, y_pred_train)
acc_test  = accuracy_score(y_test_np,  y_pred_test)

print(f"Train accuracy: {acc_train:.4f}")
print(f"Test  accuracy: {acc_test:.4f}")

## Visualisation rapide des features

In [ ]:
iterative = iter(train_eval_loader)
x_batch, _ = next(iterative)
x_batch = x_batch.to(device)
out = model(x_batch.permute(0, 2, 1))
B, _, _ = out.shape
print("out shape:", out.shape)
print("x_batch shape:", x_batch.shape)

for j in range(min(B, 4)):
    plt.figure()
    plt.imshow(out[j].squeeze().cpu().detach(), aspect="auto")
    plt.colorbar()
    plt.title(f"Feature map sample {j}")
    plt.show()

    plt.figure()
    plt.plot(x_batch[j].view(-1).cpu().detach())
    plt.title(f"Raw signal sample {j}")
    plt.show()